In [11]:
import pandas as pd
import numpy as np
from utils import utils
import functools as ft

In [12]:
df_papers = [pd.read_excel('papers_processed/papers_final.xlsx')]
df_papers.append(pd.read_excel('papers_processed/gabriel.xlsx'))
df_papers.append(pd.read_excel('papers_processed/karmanya_main_results_new.xlsx'))
df_papers.append(pd.read_excel('papers_processed/maik.xlsx'))
df_papers.append(pd.read_excel('papers_processed/Paper2_data.xlsx'))
df_papers.append(pd.read_excel('papers_processed/Paper9_data.xlsx'))
df_papers.append(pd.read_excel('papers_processed/Paper16_data.xlsx'))
df_papers.append(pd.read_excel('papers_processed/Paper18_data.xlsx'))
df_papers.append(pd.read_excel('papers_processed/Paper21_data.xlsx'))
df_papers.append(pd.read_excel('papers_processed/Paper29_data.xlsx'))
df_papers.append(pd.read_excel('papers_processed/Paper30_data.xlsx'))
df_papers.append(pd.read_excel('papers_processed/Paper33_data.xlsx'))
df_data = pd.read_csv('combined data/yearly_jrc_agri4cast_and_soil_data.csv')

df_papers

[                             doi selection type selection  \
 0   10.1016/bs.agron.2017.11.005            NaN       NaN   
 1      10.1007/s11104-018-3619-4            NaN       NaN   
 2       10.1088/1748-9326/aa67bd       latitude       <35   
 3       10.1088/1748-9326/aa67bd       latitude       >35   
 4      10.1007/s11056-015-9491-7            NaN       NaN   
 ..                           ...            ...       ...   
 61   10.1016/j.agwat.2017.11.019            NaN       NaN   
 62   10.1016/j.agwat.2018.08.035            NaN       NaN   
 63   10.1016/j.agwat.2018.07.038        soil ph       < 7   
 64   10.1016/j.agwat.2018.07.038        soil ph        >7   
 65    10.1038/s41598-018-32964-4            NaN       NaN   
 
    selection information page         farming practice  \
 0                         NaN                      NaN   
 1                         NaN                      NaN   
 2                         NaN        biochar amendment   
 3                

In [13]:
df_data.head()

,Unnamed: 0,lon,lat,yearly_rain_6,yearly_rain_7,yearly_rain_8,yearly_rain_9,yearly_rain_10,yearly_rain_11,yearly_rain_12,...,nitrogen,sand,clay,silt,cfvo,soc,oc,ocd_mean,bdod_mean,cec_mean
0,0,31.938307,26.862311,0.2,0.0,2.2,0.1,17.200000,1.5,1.0,...,66.5657,481.9009,305.9815,212.1203,263.2598,41.9028,18.2268,118.4097,133.3838,158.8610
1,1,32.255928,26.862311,0.2,0.0,2.5,0.1,17.900000,1.4,1.2,...,67.8604,479.1478,286.4404,234.4126,263.1713,45.5470,18.4873,110.5106,134.2556,166.0894
2,2,32.573549,26.862311,0.2,0.0,2.7,0.0,19.100000,0.8,1.1,...,73.3602,466.6892,275.3424,257.9714,325.3555,58.4900,17.9992,119.6809,134.8348,158.5099
3,3,32.891170,26.862311,0.2,0.0,3.0,0.0,21.600000,0.6,1.1,...,107.3417,535.1691,244.3798,220.4520,248.5240,45.1325,17.6230,120.5966,132.6689,151.6080
4,4,33.208791,26.862311,0.1,0.0,3.9,0.0,24.800001,0.4,1.2,...,89.3602,435.4464,250.0902,314.4668,351.7962,74.0498,15.5543,132.6581,135.7268,166.4694


In [14]:
def process_numerical(env_data, row, selection_env, temp):
    print(row['selection'])
    selection = row['selection']
    if "-" in selection:
        bounds = selection.split('-')
        # print(bounds)
        satisfying_rows = env_data[(env_data[selection_env] > float(bounds[0]))]
        satisfying_rows = satisfying_rows[(satisfying_rows[selection_env] < float(bounds[1]))]
        satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming practice'] = row['results_detailed'], row['results_actual'], row['farming practice']
        temp.append(satisfying_rows)
        # print(satisfying_rows)
    elif ">" in selection:
        bound = selection[1:]
        satisfying_rows = env_data[(env_data[selection_env] > float(bound))]
        satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming practice'] = row['results_detailed'], row['results_actual'], row['farming practice']
        temp.append(satisfying_rows)
    elif "<" in selection:
        bound = selection[1:]
        satisfying_rows = env_data[(env_data[selection_env] < float(bound))]
        satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming practice'] = row['results_detailed'], row['results_actual'], row['farming practice']
        temp.append(satisfying_rows)
    return temp

In [15]:
process_dict = {
    'yearly precipitation': 'precip_mean',
    'latitude': 'lat',
    'soil ph': 'ph',
    'yearly temperature': 'mean_temp_mean'
}

In [16]:
def combine(env_data, paper_data):
    temp = []
    for idx, row in paper_data.iterrows():
        sel_type = row['selection type']
        if sel_type in process_dict.keys():
            temp = process_numerical(env_data, row, process_dict[sel_type], temp)
    combined = pd.concat(temp, ignore_index=True)   
    return combined
            
                

In [17]:
df_papers[1]

,doi,selection type,selection,farming practice,farming_practice_compared,results_detailed,results_actual,crop_type,carbon_footprint,Unnamed: 9
0,10.1016/j.fcr.2015.07.023,latitude IGNORE,Tropical/Subtropical,No-till (NT) w/ N fert,Control/Conventional,crop_yield,-0.107,Others,NaN,NaN
1,10.1016/j.fcr.2015.07.023,latitude,30-60,No-till (NT) w/ N fert,Control/Conventional,crop_yield,"""-3.7""",Others,NaN,NaN
2,10.1017/S0021859614001002,latitude IGNORE,Tropical,Zero Tillage,CT,crop_yield,+7.7%,"Maize, Wheat and others",NaN,NaN
3,10.1017/S0021859614001002,latitude,30-60,Zero Tillage,CT,crop_yield,"""-3""","Maize, Wheat and others",NaN,NaN
4,10.1016/j.fcr.2015.07.020,climate,Humid vs. Dry,NT(humid),NT(dry),crop_yield,"""-12""","Maize, Wheat",NaN,NaN
5,10.1111/j.1365-2486.2012.02779.x,"climate , duration","Dry, <10 yr",NT/RT,CT,crop_yield,+57%,Others,NaN,NaN
6,10.1111/j.1365-2486.2012.02779.x,climate / duration,"Dry, >10 yr",NT/RT,CT,crop_yield,-0.27,Others,NaN,NaN
7,10.1016/j.agsy.2011.12.004,region,Global,Organic,Conventional,crop_yield,-0.2,"Maize, Wheat and other cereals",NaN,NaN
8,10.1073/pnas.1906909117,region,Simple vs. Complex Landscape,Organic,Conventional,crop_yield,-0.18,Others,NaN,NaN
9,10.1073/pnas.1906909117,region,Simple vs. Complex Landscape,Organic,Conventional,crop_yield,-0.5,Others,NaN,NaN


In [ ]:

for idx, data in enumerate(df_papers):
    df_papers[idx] = df_papers[idx].rename(columns={"results detailed":"results_detailed", "results actual":"results_actual", "detailed_result":"results_detailed", "actual_results":"results_actual","results_actual_percentage":"results_actual"})



In [19]:
df_papers[1]

,doi,selection type,selection,farming practice,farming_practice_compared,results_detailed,results_actual,crop_type,carbon_footprint,Unnamed: 9
0,10.1016/j.fcr.2015.07.023,latitude IGNORE,Tropical/Subtropical,No-till (NT) w/ N fert,Control/Conventional,crop_yield,-0.107,Others,NaN,NaN
1,10.1016/j.fcr.2015.07.023,latitude,30-60,No-till (NT) w/ N fert,Control/Conventional,crop_yield,"""-3.7""",Others,NaN,NaN
2,10.1017/S0021859614001002,latitude IGNORE,Tropical,Zero Tillage,CT,crop_yield,+7.7%,"Maize, Wheat and others",NaN,NaN
3,10.1017/S0021859614001002,latitude,30-60,Zero Tillage,CT,crop_yield,"""-3""","Maize, Wheat and others",NaN,NaN
4,10.1016/j.fcr.2015.07.020,climate,Humid vs. Dry,NT(humid),NT(dry),crop_yield,"""-12""","Maize, Wheat",NaN,NaN
5,10.1111/j.1365-2486.2012.02779.x,"climate , duration","Dry, <10 yr",NT/RT,CT,crop_yield,+57%,Others,NaN,NaN
6,10.1111/j.1365-2486.2012.02779.x,climate / duration,"Dry, >10 yr",NT/RT,CT,crop_yield,-0.27,Others,NaN,NaN
7,10.1016/j.agsy.2011.12.004,region,Global,Organic,Conventional,crop_yield,-0.2,"Maize, Wheat and other cereals",NaN,NaN
8,10.1073/pnas.1906909117,region,Simple vs. Complex Landscape,Organic,Conventional,crop_yield,-0.18,Others,NaN,NaN
9,10.1073/pnas.1906909117,region,Simple vs. Complex Landscape,Organic,Conventional,crop_yield,-0.5,Others,NaN,NaN


In [20]:
combined = []
for idx, data in enumerate(df_papers):
    print("---------------------------------------------")
    print(idx)
    combined.append(combine(df_data, data))
combined

---------------------------------------------
0
<35
>35
<7 
>7 
<200 
200-400
400-800
>800
>12
<12
<200
200-400
400-800
>800
>9
<9
>9
<7
>7
<7
>7
< 7
>7
---------------------------------------------
1
30-60
30-60
30-60
---------------------------------------------
2
250–400
400–500
500–600
>600
<10
10–20
10–20
>20
200–300
>300
<200
---------------------------------------------
3
30-60
30-60
<13
>13
<400
400–800
<6.5
6.5–7.5
>7.5
<6.5
6.5–7.5
>7.5
---------------------------------------------
4


/var/folders/79/r507kdn15rd10yj364f325mm0000gn/T/ipykernel_49546/2369048401.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming practice'] = row['results_detailed'], row['results_actual'], row['farming practice']
/var/folders/79/r507kdn15rd10yj364f325mm0000gn/T/ipykernel_49546/2369048401.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  satisfying_rows['results_detailed'], satisfying_rows['results_actual'], 

KeyError: 'selection type'

In [ ]:
c = combine(df_data, df_papers)
c

In [ ]:
c.to_csv('example.csv', index=False)